# <center> Wavefront set extractor using tensorflow high order Ellipses and Parallelogram</center>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import dense.shearlab as shearlab
import dense.batchgen as bg
import matplotlib.pyplot as plt
%matplotlib inline
import time

In [3]:
size = 1024
nClasses = 90
nDistr = 20
size_patch = 21

In [4]:
rows = size
cols = size
nScales = 4
shearletSystem = shearlab.getshearletsystem2D(rows,cols,nScales);

In [5]:
# Lets create several data

In [6]:
array1, label1 = bg.gen_batch_smooth(size, nClasses, nDistr, shearletSystem, size_patch)

In [7]:
array2, label2 = bg.gen_batch_smooth(size, nClasses, nDistr, shearletSystem, size_patch)

In [8]:
array3, label3 = bg.gen_batch_smooth(size, nClasses, nDistr, shearletSystem, size_patch)

In [9]:
array4, label4 = bg.gen_batch_smooth(size, nClasses, nDistr, shearletSystem, size_patch)

In [10]:
array5, label5 = bg.gen_batch_smooth(size, nClasses, nDistr, shearletSystem, size_patch)

In [11]:
array6, label6 = bg.gen_batch_smooth(size, nClasses, nDistr, shearletSystem, size_patch)

In [12]:
array7, label7 = bg.gen_batch_smooth(size, nClasses, nDistr, shearletSystem, size_patch)

In [13]:
array8, label8 = bg.gen_batch_smooth(size, nClasses, nDistr, shearletSystem, size_patch)

Concatenate the arrays and labels

In [14]:
array = np.concatenate((array1,array2,array3,array4, array5, array6, array8))
label = np.concatenate((label1,label2,label3,label4, label5, label6, label8))

In [5]:
import h5py

**Saving the dataset**

In [16]:
# create an hdf5 file
with h5py.File("array512ellipsesparalel_highorder.h5") as f:
    # create a dataset for your movie
    dst = f.create_dataset("array", shape=(array.shape[0], array.shape[1], 
                                           array.shape[2],array.shape[3]),
                           dtype=np.float64)
    for frame in range(array.shape[0]):
        dst[frame] = array[frame,:,:,:]

In [17]:
# create an hdf5 file
with h5py.File("label512ellipsesparalel_highorder.h5") as f:
    # create a dataset for your movie
    dst_label = f.create_dataset("label", shape=(label.shape[0], label.shape[1]),
                           dtype=np.float64)
    for frame in range(label.shape[0]):
        dst_label[frame] = label[frame,:]

**Loading dataset**

In [9]:
#Reading the data
with h5py.File("array512ellipsesparalel_highorder.h5", 'r') as h5:
    array = h5["array"][:]

In [19]:
#Reading the data
with h5py.File("label512ellipsesparalel_highorder.h5", 'r') as h5:
    label = h5["label"][:]

Do some random permutations

In [20]:
permutations = np.random.permutation(array.shape[0])

In [21]:
array = array[permutations,:,:,:]
label = label[permutations,:]

In [22]:
array.shape

(187684, 20, 20, 49)

In [23]:
label.shape

(187684, 91)

### Classifying the second angle

Lets create several data

In [24]:
def extract_angle(label,angle):
    return label[angle] == 1.0

In [25]:
def generate_training_test(array, label, angle):
    # Extract the position of the elements with that angle
    labels_angle = [extract_angle(labeli,angle) for labeli in label]
    # Generating the labels and arrays
    label_angle = np.ones(sum(labels_angle))
    array_angle = array[labels_angle]
    label_noangle = np.zeros(sum(labels_angle))
    array_noangle = array[~np.array(labels_angle)]
    # Subsampling the arrays without that angle
    array_noangle = array_noangle[0:sum(labels_angle)]
    # Creating the test, training and validation
    X = np.concatenate((array_angle,array_noangle))
    y = np.concatenate((label_angle,label_noangle))
    # Permutations
    permutations = np.random.permutation(X.shape[0])
    X = X[permutations]
    y = y[permutations]
    # Train test
    X_train = X[0:int(X.shape[0]*0.8-1)]
    y_train = y[0:int(y.shape[0]*0.8-1)]
    # Test set
    X_test = X[int(X.shape[0]*0.8):(X.shape[0]-1)]
    y_test = y[int(y.shape[0]*0.8):(y.shape[0]-1)]
    # Now validation
    X_valid = X_train[int(X_train.shape[0]*0.8):(X_train.shape[0]-1)]
    y_valid = y_train[int(y_train.shape[0]*0.8):(y_train.shape[0]-1)]
    X_train = X_train[0:int(X_train.shape[0]*0.8-1)]
    y_train = y_train[0:int(y_train.shape[0]*0.8-1)]
    
    return X_train, X_test, X_valid, y_train, y_test, y_valid

In [26]:
label.shape

(187684, 91)

In [27]:
# Classification of second angle
angle = 1

In [28]:
X_train, X_test, X_valid, y_train, y_test, y_valid = generate_training_test(array, label, angle)

In [29]:
X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape

((2990, 20, 20, 49),
 (935, 20, 20, 49),
 (747, 20, 20, 49),
 (2990,),
 (935,),
 (747,))

### Save data sets

In [30]:
# create an hdf5 file
with h5py.File('angle2ellipsesparalel_highorder.h5') as f:
    # create a dataset for your movie
    X_trainf = f.create_dataset("X_train", shape=(X_train.shape[0], X_train.shape[1], 
                                           X_train.shape[2],X_train.shape[3]),
                           dtype=np.float64)
    for frame in range(X_train.shape[0]):
        X_trainf[frame] = X_train[frame,:,:,:]
        
    X_validf = f.create_dataset("X_valid", shape=(X_valid.shape[0], X_valid.shape[1], 
                                           X_valid.shape[2],X_valid.shape[3]),
                           dtype=np.float64)
    for frame in range(X_valid.shape[0]):
        X_validf[frame] = X_valid[frame,:,:,:]
    
    X_testf = f.create_dataset("X_test", shape=(X_test.shape[0], X_test.shape[1], 
                                                 X_test.shape[2],X_test.shape[3]),
                           dtype=np.float64)
    for frame in range(X_test.shape[0]):
        X_testf[frame] = X_test[frame,:,:,:]
    
    y_trainf = f.create_dataset("y_train", shape=(y_train.shape[0],),
                           dtype=np.float64)
    for frame in range(y_train.shape[0]):
        y_trainf[frame] = y_train[frame]
    
    y_validf = f.create_dataset("y_valid", shape=(y_valid.shape[0],),
                           dtype=np.float64)
    for frame in range(y_valid.shape[0]):
        y_validf[frame] = y_valid[frame]

    y_testf = f.create_dataset("y_test", shape=(y_test.shape[0],),
                           dtype=np.float64)
    for frame in range(y_test.shape[0]):
        y_testf[frame] = y_test[frame]

### Load dataset

In [6]:
#Reading the data
with h5py.File("angle2ellipsesparalel_highorder.h5", 'r') as h5:
    X_train = h5["X_train"][:]
    X_test = h5["X_test"][:]
    X_valid = h5["X_valid"][:]
    y_train = h5["y_train"][:]
    y_test = h5["y_test"][:]
    y_valid = h5["y_valid"][:]

In [7]:
X_test = X_test.astype('float32')
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
y_test = y_test.astype('float32')
y_train = y_train.astype('float32')
y_valid = y_valid.astype('float32')

In [8]:
 X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape

((2990, 20, 20, 49),
 (935, 20, 20, 49),
 (747, 20, 20, 49),
 (2990,),
 (935,),
 (747,))

## Training

Lets create an architecture

In [9]:
### Training the model

In [10]:
from __future__ import division, print_function, absolute_import

In [11]:
from adler.tensorflow import prelu, cosine_decay

In [12]:
import os
import adler
adler.util.gpu.setup_one_gpu()
import warnings
warnings.filterwarnings('ignore')

Picking GPU 0


In [13]:
import tensorflow as tf
import numpy as np

In [14]:
from tensorflow.keras.utils import to_categorical

In [15]:
np.random.seed(0)
#name = os.path.splitext(os.path.basename(__file__))[0]
name = os.path.splitext(os.getcwd())[0]

**To categorical**

In [16]:
y_train = to_categorical(y_train, num_classes = 2)
y_test = to_categorical(y_test, num_classes = 2)
y_valid = to_categorical(y_valid, num_classes = 2)

In [17]:
width = X_train.shape[1]
height = X_train.shape[2]
channels = X_train.shape[3]
nLabel = 2

In [18]:
## Weight Initialization
# Create lots of weights and biases & Initialize with a small positive number as we will use ReLU
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

## Convolution and Pooling
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME') 

## Pooling: max pooling over 2x2 blocks
def max_pool_2x2(x): 
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

**Architecture**: 4 convolutional layers and 2 fully connected

**Model**

In [19]:
conv_layers = 4
fully_connected = 1

In [20]:
graph = tf.Graph()
with graph.as_default():
    # Placeholders
    x = tf.placeholder(tf.float32, shape=(None, width, height,channels))
    y_ = tf.placeholder(tf.float32, shape=(None, nLabel))
    
    ## First Convolutional Layer
    W_conv1 = weight_variable([3, 3, 49, 49*4])
    b_conv1 = bias_variable([49*4])
    #Convolution
    h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
    print('h_conv1',h_conv1.shape)
    
    # Batch normalization
    # Calculate batch mean and variance
    batch_mean1, batch_var1 = tf.nn.moments(h_conv1,[0])
    h_conv1hat = (h_conv1-batch_mean1) / tf.sqrt(batch_var1 + 1e-3)
    # Pooling
    #h_pool1 = max_pool_2x2(h_conv1hat) 
    #print('h_pool1',h_pool1.shape)
    # No_pooling
    h_pool1 = h_conv1hat
    print('h_pool1',h_pool1.shape)
    
    ## Second Convolutional Layer
    W_conv2 = weight_variable([3, 3, 49*4, 49*4*4])
    b_conv2 = bias_variable([49*4*4])
    #Convolution
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    print('h_conv2',h_conv2.shape)
    
    # Batch normalization
    # Calculate batch mean and variance
    batch_mean2, batch_var2 = tf.nn.moments(h_conv2,[0])
    h_conv2hat = (h_conv2-batch_mean2) / tf.sqrt(batch_var2 + 1e-3)
    
    # Pooling
    h_pool2 = max_pool_2x2(h_conv2hat) 
    print('h_pool2',h_pool2.shape)
    
    ## Third Convolutional Layer
    W_conv3 = weight_variable([3, 3, 49*4*4, 49*4*4*2])
    b_conv3 = bias_variable([49*4*4*2])
    #Convolution
    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
    print('h_conv3',h_conv3.shape)
    
    # Batch normalization
    # Calculate batch mean and variance
    batch_mean3, batch_var3 = tf.nn.moments(h_conv3,[0])
    h_conv3hat = (h_conv3-batch_mean3) / tf.sqrt(batch_var3 + 1e-3)
    
    # Pooling
    h_pool3 = max_pool_2x2(h_conv3hat) 
    print('h_pool3',h_pool3.shape)

    
    ## Third Convolutional Layer
    W_conv4 = weight_variable([3, 3, 49*4*4*2, 49*4*4*2*2])
    b_conv4 = bias_variable([49*4*4*2*2])
    #Convolution
    h_conv4 = tf.nn.relu(conv2d(h_pool3, W_conv4) + b_conv4)
    print('h_conv4',h_conv4.shape)
    
    # Batch normalization
    # Calculate batch mean and variance
    batch_mean4, batch_var4 = tf.nn.moments(h_conv4,[0])
    h_conv4hat = (h_conv4-batch_mean4) / tf.sqrt(batch_var4 + 1e-3)
    
    # Pooling
    h_pool4 = max_pool_2x2(h_conv4hat) 
    print('h_pool4',h_pool4.shape)

    ## Densely Connected Layer 

    # new shapes of pooled vectors
    _, width_pooled, height_pooled, channels_pooled = h_pool4.shape

    # fully-connected layer with 1024 neurons to process on the entire image
    W_fc1 = weight_variable([int(width_pooled*height_pooled*channels_pooled), 1024])  
    b_fc1 = bias_variable([1024])
    
    # Flat the output of the convolutional labels
    h_pool4_flat = tf.reshape(h_pool4, [-1, int(width_pooled*height_pooled*channels_pooled)])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool4_flat, W_fc1) + b_fc1)
    
    print('h_pool4_flat',h_pool4_flat.shape)

    ## Dropout (to reduce overfitting; useful when training very large neural network)
    # We will turn on dropout during training & turn off during testing
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    ## Readout Layer

    W_fc2 = weight_variable([1024, nLabel]) # [1024, 10]
    b_fc2 = bias_variable([nLabel]) # [10]
    
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

    # set up for optimization (optimizer:ADAM)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)  # 1e-4
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

h_conv1 (?, 20, 20, 196)
h_pool1 (?, 20, 20, 196)
h_conv2 (?, 20, 20, 784)
h_pool2 (?, 10, 10, 784)
h_conv3 (?, 10, 10, 1568)
h_pool3 (?, 5, 5, 1568)
h_conv4 (?, 5, 5, 3136)
h_pool4 (?, 3, 3, 3136)
h_pool4_flat (?, 28224)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



### Train the model


In [21]:
batch_size_train = 86
batch_size_valid = 1000
batch_size_test = 1000
num_steps = 5000

In [22]:
test_accuracy_minibatches = []
valid_accuracy_minibatches = []
best_validation_accuracy = 0.0
best_validation_accuracy_step = 0

In [24]:
with tf.Session(graph=graph) as session: 
    tf.global_variables_initializer().run()
    print('Initialized')
    
    # Saver setup
    saver = tf.train.Saver()
    save_dir = 'checkpoints_ellipsesparalel_highorder/'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_path = os.path.join(save_dir, 'model_angle2')

    
    # Train the model
    for step in range(num_steps):
        # Logging every 100th iteration in the training process.
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size_train) % (y_train.shape[0] - batch_size_train)
        
        ## Lets do validation
        offset_valid = (step * batch_size_valid) % (y_valid.shape[0] - batch_size_valid)

        # Generate a minibatch.
        batch_data = X_train[offset:(offset + batch_size_train), :]
        batch_labels = y_train[offset:(offset + batch_size_train), :]
        if step%5 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch_data, y_: batch_labels, keep_prob: 1.0})
            print("step %d, training accuracy %g"%(step, train_accuracy))
            
        if step%100 == 0:
            ## Some test
            # Generate a test minibatch.
            batch_data_valid = X_valid[offset_valid:(offset_valid + batch_size_valid), :]
            batch_labels_valid = y_valid[offset_valid:(offset_valid + batch_size_valid), :]
            valid_accuracy = accuracy.eval(feed_dict={x:batch_data_valid, y_: batch_labels_valid, keep_prob: 1.0})
            valid_accuracy_minibatches.append(valid_accuracy)
            print("\033[1m step %d, validation accuracy \033[0;0m %g"%(step, valid_accuracy))
            
            # Save when there is an improvement 
            if valid_accuracy > best_validation_accuracy:
                print("\033[1m The validation accuracy has improved from the last checkpoint \033[0;0m")
                best_validation_accuracy = valid_accuracy
                best_validation_accuracy_step = step
                saver.save(sess = session, save_path=save_path)
            else:
                print("\033[1m The validation accuracy DID NOT improved from the last checkpoint \033[0;0m")

            
        train_step.run(feed_dict={x: batch_data, y_: batch_labels, keep_prob: 0.5})
        
        
    # Evaulate our accuracy on the test data
    # Test the model
    for step in range(num_steps):
        offset_test = (step * batch_size_test) % (y_test.shape[0] - batch_size_test)

        # Generate a minibatch.
        batch_data_test = X_test[offset_test:(offset_test + batch_size_test), :]
        batch_labels_test = y_test[offset_test:(offset_test + batch_size_test), :]
        test_accuracy = accuracy.eval(feed_dict={x:batch_data_test, y_: batch_labels_test, keep_prob: 1.0})
        test_accuracy_minibatches.append(test_accuracy)
        
        if step%5 == 0:
            print("step %d, test accuracy %g"%(step, test_accuracy))

Initialized
step 0, training accuracy 0.511628
 step 0, validation accuracy  0.49
 The validation accuracy has improved from the last checkpoint 
step 5, training accuracy 0.593023
step 10, training accuracy 0.488372
step 15, training accuracy 0.534884
step 20, training accuracy 0.534884
step 25, training accuracy 0.593023
step 30, training accuracy 0.465116
step 35, training accuracy 0.662791
step 40, training accuracy 0.616279
step 45, training accuracy 0.546512
step 50, training accuracy 0.55814
step 55, training accuracy 0.639535
step 60, training accuracy 0.593023
step 65, training accuracy 0.534884
step 70, training accuracy 0.511628
step 75, training accuracy 0.55814
step 80, training accuracy 0.639535
step 85, training accuracy 0.732558
step 90, training accuracy 0.709302
step 95, training accuracy 0.604651
step 100, training accuracy 0.697674
 step 100, validation accuracy  0.668
 The validation accuracy has improved from the last checkpoint 
step 105, training accuracy 0.6279

step 950, training accuracy 0.930233
step 955, training accuracy 0.94186
step 960, training accuracy 0.918605
step 965, training accuracy 0.94186
step 970, training accuracy 0.965116
step 975, training accuracy 0.976744
step 980, training accuracy 0.906977
step 985, training accuracy 0.953488
step 990, training accuracy 0.965116
step 995, training accuracy 0.965116
step 1000, training accuracy 0.976744
 step 1000, validation accuracy  0.956
 The validation accuracy DID NOT improved from the last checkpoint 
step 1005, training accuracy 0.94186
step 1010, training accuracy 0.976744
step 1015, training accuracy 0.930233
step 1020, training accuracy 0.94186
step 1025, training accuracy 0.94186
step 1030, training accuracy 0.94186
step 1035, training accuracy 0.965116
step 1040, training accuracy 0.918605
step 1045, training accuracy 0.965116
step 1050, training accuracy 0.930233
step 1055, training accuracy 0.953488
step 1060, training accuracy 0.988372
step 1065, training accuracy 0.9418

step 1890, training accuracy 0.976744
step 1895, training accuracy 0.988372
step 1900, training accuracy 0.988372
 step 1900, validation accuracy  0.956
 The validation accuracy DID NOT improved from the last checkpoint 
step 1905, training accuracy 0.94186
step 1910, training accuracy 0.965116
step 1915, training accuracy 0.965116
step 1920, training accuracy 0.988372
step 1925, training accuracy 0.965116
step 1930, training accuracy 0.965116
step 1935, training accuracy 0.94186
step 1940, training accuracy 0.930233
step 1945, training accuracy 0.918605
step 1950, training accuracy 0.953488
step 1955, training accuracy 0.988372
step 1960, training accuracy 0.953488
step 1965, training accuracy 0.953488
step 1970, training accuracy 0.953488
step 1975, training accuracy 0.965116
step 1980, training accuracy 0.976744
step 1985, training accuracy 0.930233
step 1990, training accuracy 0.976744
step 1995, training accuracy 0.965116
step 2000, training accuracy 0.953488
 step 2000, validatio

step 2810, training accuracy 0.976744
step 2815, training accuracy 0.94186
step 2820, training accuracy 0.965116
step 2825, training accuracy 0.965116
step 2830, training accuracy 0.965116
step 2835, training accuracy 0.953488
step 2840, training accuracy 0.976744
step 2845, training accuracy 0.976744
step 2850, training accuracy 0.988372
step 2855, training accuracy 0.965116
step 2860, training accuracy 0.976744
step 2865, training accuracy 0.965116
step 2870, training accuracy 0.988372
step 2875, training accuracy 0.976744
step 2880, training accuracy 0.953488
step 2885, training accuracy 0.953488
step 2890, training accuracy 0.965116
step 2895, training accuracy 0.988372
step 2900, training accuracy 0.976744
 step 2900, validation accuracy  0.969
 The validation accuracy DID NOT improved from the last checkpoint 
step 2905, training accuracy 0.94186
step 2910, training accuracy 0.965116
step 2915, training accuracy 0.965116
step 2920, training accuracy 0.965116
step 2925, training a

step 3755, training accuracy 1
step 3760, training accuracy 0.94186
step 3765, training accuracy 0.988372
step 3770, training accuracy 0.988372
step 3775, training accuracy 0.976744
step 3780, training accuracy 0.930233
step 3785, training accuracy 0.965116
step 3790, training accuracy 0.965116
step 3795, training accuracy 1
step 3800, training accuracy 0.930233
 step 3800, validation accuracy  0.957
 The validation accuracy DID NOT improved from the last checkpoint 
step 3805, training accuracy 0.988372
step 3810, training accuracy 0.94186
step 3815, training accuracy 0.953488
step 3820, training accuracy 0.976744
step 3825, training accuracy 0.965116
step 3830, training accuracy 0.953488
step 3835, training accuracy 0.976744
step 3840, training accuracy 0.988372
step 3845, training accuracy 0.965116
step 3850, training accuracy 0.94186
step 3855, training accuracy 0.988372
step 3860, training accuracy 1
step 3865, training accuracy 0.965116
step 3870, training accuracy 0.953488
step 

step 4700, training accuracy 0.953488
 step 4700, validation accuracy  0.975
 The validation accuracy DID NOT improved from the last checkpoint 
step 4705, training accuracy 0.94186
step 4710, training accuracy 0.94186
step 4715, training accuracy 0.965116
step 4720, training accuracy 0.965116
step 4725, training accuracy 0.976744
step 4730, training accuracy 0.988372
step 4735, training accuracy 0.965116
step 4740, training accuracy 0.965116
step 4745, training accuracy 0.988372
step 4750, training accuracy 0.965116
step 4755, training accuracy 0.930233
step 4760, training accuracy 0.953488
step 4765, training accuracy 0.965116
step 4770, training accuracy 1
step 4775, training accuracy 0.988372
step 4780, training accuracy 0.965116
step 4785, training accuracy 0.965116
step 4790, training accuracy 0.976744
step 4795, training accuracy 0.930233
step 4800, training accuracy 0.976744
 step 4800, validation accuracy  0.952
 The validation accuracy DID NOT improved from the last checkpoin

step 935, test accuracy 0.983
step 940, test accuracy 0.975
step 945, test accuracy 0.985
step 950, test accuracy 0.975
step 955, test accuracy 0.984
step 960, test accuracy 0.974
step 965, test accuracy 0.984
step 970, test accuracy 0.976
step 975, test accuracy 0.983
step 980, test accuracy 0.977
step 985, test accuracy 0.984
step 990, test accuracy 0.977
step 995, test accuracy 0.971
step 1000, test accuracy 0.977
step 1005, test accuracy 0.971
step 1010, test accuracy 0.975
step 1015, test accuracy 0.971
step 1020, test accuracy 0.977
step 1025, test accuracy 0.974
step 1030, test accuracy 0.975
step 1035, test accuracy 0.975
step 1040, test accuracy 0.976
step 1045, test accuracy 0.977
step 1050, test accuracy 0.979
step 1055, test accuracy 0.976
step 1060, test accuracy 0.982
step 1065, test accuracy 0.973
step 1070, test accuracy 0.984
step 1075, test accuracy 0.974
step 1080, test accuracy 0.981
step 1085, test accuracy 0.972
step 1090, test accuracy 0.98
step 1095, test accura

step 2265, test accuracy 0.973
step 2270, test accuracy 0.981
step 2275, test accuracy 0.972
step 2280, test accuracy 0.981
step 2285, test accuracy 0.973
step 2290, test accuracy 0.98
step 2295, test accuracy 0.975
step 2300, test accuracy 0.981
step 2305, test accuracy 0.978
step 2310, test accuracy 0.981
step 2315, test accuracy 0.977
step 2320, test accuracy 0.983
step 2325, test accuracy 0.977
step 2330, test accuracy 0.984
step 2335, test accuracy 0.975
step 2340, test accuracy 0.985
step 2345, test accuracy 0.975
step 2350, test accuracy 0.984
step 2355, test accuracy 0.974
step 2360, test accuracy 0.983
step 2365, test accuracy 0.977
step 2370, test accuracy 0.983
step 2375, test accuracy 0.977
step 2380, test accuracy 0.983
step 2385, test accuracy 0.976
step 2390, test accuracy 0.971
step 2395, test accuracy 0.977
step 2400, test accuracy 0.971
step 2405, test accuracy 0.977
step 2410, test accuracy 0.969
step 2415, test accuracy 0.976
step 2420, test accuracy 0.974
step 2425

step 3590, test accuracy 0.969
step 3595, test accuracy 0.976
step 3600, test accuracy 0.971
step 3605, test accuracy 0.977
step 3610, test accuracy 0.973
step 3615, test accuracy 0.976
step 3620, test accuracy 0.976
step 3625, test accuracy 0.976
step 3630, test accuracy 0.978
step 3635, test accuracy 0.979
step 3640, test accuracy 0.976
step 3645, test accuracy 0.981
step 3650, test accuracy 0.974
step 3655, test accuracy 0.983
step 3660, test accuracy 0.974
step 3665, test accuracy 0.982
step 3670, test accuracy 0.973
step 3675, test accuracy 0.98
step 3680, test accuracy 0.973
step 3685, test accuracy 0.981
step 3690, test accuracy 0.975
step 3695, test accuracy 0.981
step 3700, test accuracy 0.978
step 3705, test accuracy 0.981
step 3710, test accuracy 0.977
step 3715, test accuracy 0.982
step 3720, test accuracy 0.977
step 3725, test accuracy 0.984
step 3730, test accuracy 0.974
step 3735, test accuracy 0.985
step 3740, test accuracy 0.975
step 3745, test accuracy 0.985
step 3750

step 4915, test accuracy 0.983
step 4920, test accuracy 0.975
step 4925, test accuracy 0.985
step 4930, test accuracy 0.975
step 4935, test accuracy 0.984
step 4940, test accuracy 0.974
step 4945, test accuracy 0.984
step 4950, test accuracy 0.976
step 4955, test accuracy 0.983
step 4960, test accuracy 0.977
step 4965, test accuracy 0.984
step 4970, test accuracy 0.977
step 4975, test accuracy 0.971
step 4980, test accuracy 0.977
step 4985, test accuracy 0.971
step 4990, test accuracy 0.975
step 4995, test accuracy 0.971


In [1]:
print("The average test accuracy is "+str(sum(test_accuracy_minibatches)/len(test_accuracy_minibatches)*100))

The average test accuracy is 93.36999445
